## Finetuning Mistral на диалоговом датасете

In [ ]:
!pip install trl

  Using cached transformers-4.52.1-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.52.1-py3-none-any.whl (10.5 MB)


In [ ]:
import json
import torch
from datasets import Dataset
from peft import LoraConfig
from sklearn.model_selection import train_test_split
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

**Загружаем датасет**

https://pineforest-ai.com/wp-content/uploads/llms_course/company_cases.zip


In [ ]:
with open("/content/company_cases.json", 'r') as inp:
    raw_dataset = json.load(inp)

train_raw_dataset, test_raw_dataset = train_test_split(raw_dataset)
train_test_raw_dataset = {"train": train_raw_dataset, "test": test_raw_dataset}

**Преобразуем датасет в формат, который используется в классе Dataset**

In [ ]:
raw_dataset_dict = {}
for data_type in ["train", "test"]:
    raw_dataset_dict[data_type] = {
        "instruction": [element['instruction'] for element in train_test_raw_dataset[data_type]],
        "personality": [element['personality'] for element in train_test_raw_dataset[data_type]],
        "context": [element['context'] for element in train_test_raw_dataset[data_type]],
        "dialog_start_line": [element['instruction'] for element in train_test_raw_dataset[data_type]],
        "dialog": [element['dialog'] for element in train_test_raw_dataset[data_type]]
    }

train_dataset = Dataset.from_dict(raw_dataset_dict["train"])
test_dataset = Dataset.from_dict(raw_dataset_dict["test"])

**Функция для форматирования промпта**

In [ ]:
def formatting_prompts_func(examples):
    prompts = []
    for i in range(len(examples['instruction'])):
        prompt = f"<s>system\n{examples['instruction'][i]}"
        if examples['personality'][i]:
            prompt += f"\n{examples['personality'][i]}"
        prompt += f"\n{examples['context'][i]}"
        if examples['dialog_start_line']:
            prompt += f"\n{examples['dialog_start_line'][i]}"
        prompt += "</s>"
        for element in examples['dialog'][i]:
            prompt += f"<s>{element['role']}\n{element['content']}</s>"
        prompts.append(prompt)
    return prompts

**Инициализируем токенизатор**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga_mistral_7b")
tokenizer.pad_token_id = 0

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Сделаем, чтобы **градиент при обучении протекал только через токены последней реплики чат-бота**, labels у остальных токенов сделаем равными -100.

In [ ]:
response_template = "bot\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

**Инициализируем модель**

In [ ]:
!pip uninstall bitsandbytes transformers -y
!pip install -U git+https://github.com/TimDettmers/bitsandbytes.git
!pip install -U transformers accelerate

from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

try:
    model = AutoModelForCausalLM.from_pretrained(
        "IlyaGusev/saiga_mistral_7b",
        device_map="auto",
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        ),
        trust_remote_code=True
    )
    print("Модель успешно загружена в 4-bit режиме")

except Exception as e:
    print(f"Ошибка 4-bit: {str(e)}")
    print("Пробуем загрузку без квантования...")

    try:
        model = AutoModelForCausalLM.from_pretrained(
            "IlyaGusev/saiga_mistral_7b",
            device_map="auto",
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
        print("Модель загружена без квантования")
    except Exception as e:
        print(f"Финальная ошибка: {str(e)}")
        print("Попробуйте использовать другой GPU (A100) или уменьшить размер модели")

Found existing installation: bitsandbytes 0.45.5
Uninstalling bitsandbytes-0.45.5:
  Successfully uninstalled bitsandbytes-0.45.5
Found existing installation: transformers 4.52.1
Uninstalling transformers-4.52.1:
  Successfully uninstalled transformers-4.52.1
  Cloning https://github.com/TimDettmers/bitsandbytes.git to /tmp/pip-req-build-xrruq27v
  Running command git clone --filter=blob:none --quiet https://github.com/TimDettmers/bitsandbytes.git /tmp/pip-req-build-xrruq27v
  Resolved https://github.com/TimDettmers/bitsandbytes.git to commit cdcae8d34dcb7f67772a09331c13f783853b59d3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for bitsandbytes: filename=bitsandbytes-0.46.0.dev0-cp311-cp311-linux_x86_64.whl size=97677 sha256=fbb98a7466fb584768b317a795dc5e31f8d97c6d53223037efe422b2b9c0c6c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-s_59gj0g/wheels/85/06/31/de7992855a6f6df8812a

ERROR:bitsandbytes.cextension:bitsandbytes library load error: Configured CUDA binary not found at /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cuda124.so
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 287, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 267, in get_native_library
    raise RuntimeError(f"Configured CUDA binary not found at {cuda_binary_path}")
RuntimeError: Configured CUDA binary not found at /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cuda124.so


✅ bitsandbytes 0.46.0.dev0 успешно импортирован


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

❌ Ошибка 4-bit: 
🚨 Forgot to compile the bitsandbytes library? 🚨
1. You're not using the package but checked-out the source code
2. You MUST compile from source

Attempted to use bitsandbytes native library functionality but it's not available.

This typically happens when:
1. bitsandbytes doesn't ship with a pre-compiled binary for your CUDA version
2. The library wasn't compiled properly during installation from source

To make bitsandbytes work, the compiled library version MUST exactly match the linked CUDA version.
If your CUDA version doesn't have a pre-compiled binary, you MUST compile from source.

You have two options:
1. COMPILE FROM SOURCE (required if no binary exists):
   https://huggingface.co/docs/bitsandbytes/main/en/installation#cuda-compile
2. Use BNB_CUDA_VERSION to specify a DIFFERENT CUDA version from the detected one, which is installed on your machine and matching an available pre-compiled version listed above

Original error: Configured CUDA binary not found at 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

✅ Модель загружена без квантования


**Параметры адаптеров**: $r$ - размерность матрицы адаптеров, $lora\_alpha$ - это значение, на которое после умножения вектора на матрицу-адаптер мы будем умножать каждую компоненту полученного вектора.

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
!pip install -U transformers>=4.35.0

**Инициализируем Trainer и запускаем обучение.**


In [ ]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, tokenizer, original_data, max_length=2048):
        self.tokenizer = tokenizer
        self.texts = [self.format_dialog(d['dialog']) for d in original_data]
        self.max_length = max_length

    def format_dialog(self, dialog):
        return "\n".join([f"{turn['role']}: {turn['content']}" for turn in dialog])

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {key: val.squeeze() for key, val in encoding.items()}

tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga_mistral_7b")
tokenizer.pad_token = tokenizer.eos_token

train_dataset = TextDataset(tokenizer, train_dataset)
eval_dataset = TextDataset(tokenizer, test_dataset)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

#Проверка
sample = train_dataset[0]
print("Пример данных:", {k: v.shape for k, v in sample.items()})

<ipython-input-26-71e3fe453134>:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Пример данных: {'input_ids': torch.Size([2048]), 'attention_mask': torch.Size([2048])}


In [ ]:
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")

('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/tokenizer.model',
 './final_model/added_tokens.json',
 './final_model/tokenizer.json')